In [10]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')


In [11]:
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/mdg_south')
mofuss_regions0 = ee.FeatureCollection("users/aghilardi/mofuss_regions0_simp")
zmb = mofuss_regions0.filter(ee.Filter.eq("GID_0", "ZMB")).geometry()
roi = zmb

In [12]:
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(zmb)
Map.addLayer(zmb,{})
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
# Capturar el polígono dibujado como ROI
# roi = Map.user_roi
# coords = roi.coordinates()

# print(coords.getInfo())

# geom = ee.Geometry.Polygon(coords)
# roi = geom

In [21]:
# ---------------------------------------------------------------------
# 1) PARAMETERS
# ---------------------------------------------------------------------
EMB_COLLECTION  = 'GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL'
GEDI_COLLECTION = 'LARSE/GEDI/GEDI04_A_002_MONTHLY'
TARGET_BAND     = 'agbd'   # above-ground biomass density (Mg ha⁻¹)

# ---------------------------------------------------------------------
# 2) YEAR 1: 2017
# ---------------------------------------------------------------------
YEAR1 = 2019
START1 = ee.Date.fromYMD(YEAR1, 1, 1)
END1   = START1.advance(1, 'year')

embedding1 = (ee.ImageCollection(EMB_COLLECTION)
              .filterDate(START1, END1))

embedding_image1 = embedding1.median().clip(roi)

# ---------------------------------------------------------------------
# 3) YEAR 2: 2024
# ---------------------------------------------------------------------
YEAR2 = 2024
START2 = ee.Date.fromYMD(YEAR2, 1, 1)
END2   = START2.advance(1, 'year')

embedding2 = (ee.ImageCollection(EMB_COLLECTION)
              .filterDate(START2, END2))

embedding_image2 = embedding2.median().clip(roi)

In [22]:
Map.addLayer(embedding_image1,{},"embedding_1")
Map.addLayer(embedding_image2,{},"embedding2")

Map

Map(bottom=9044.0, center=[-12.039320557540572, 27.817382812500004], controls=(WidgetControl(options=['positio…

In [7]:
def mask_based_on_label_values(image, values):
    # Cargar la colección de DynamicWorld
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
        .filterBounds(roi) \
        .filterDate(START1, END1) \
        .select('label') \
        .median() \
        .clip(roi)
    
    # Crear una máscara basada en los valores de la lista
    label_mask = dynamic_world.eq(values[0])  # Inicializar con el primer valor
    for value in values[1:]:
        label_mask = label_mask.Or(dynamic_world.eq(value))  # Añadir los demás valores
    
    # Aplicar la máscara de valores al conjunto de datos
    return image.updateMask(label_mask)

# Ejemplo: filtrar GEDI según árboles (1), agua (5), y vegetación alta (2)
values_to_mask = [0,1,2,3,4,5,6,7,8] 


def get_gedi_data(band_name):
    return ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
        .filterBounds(roi) \
        .filterDate(START1, END1) \
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)  # Filtrar donde degrade_flag es 0
            .And(image.select('l2_quality_flag').eq(1))  # Filtrar donde l2_quality_flag es 1
            .And(image.select('l4_quality_flag').eq(1))  # Filtrar donde l4_quality_flag es 1
        )) \
        .select(band_name) \
        .median() \
        .toFloat() \
        .clip(roi)

# Obtener las bandas agbd, agbd_se y l4_quality_flag
gediData_agbd = get_gedi_data('agbd').rename('agbd')
gediData_agbd_se = get_gedi_data('agbd_se').rename('agbd_se')
gedi_combined = ee.Image.cat([gediData_agbd, gediData_agbd_se])
gedi_masked = mask_based_on_label_values(gedi_combined, values_to_mask)

# Convertir a float si es necesario
def convert_to_float(image):
    return image.float()

# Convertir la imagen GEDI enmascarada a float
gedi_masked = convert_to_float(gedi_masked)

# Puedes agregar las bandas filtradas de GEDI a tu imagen original si lo necesitas
image = embedding_image1.addBands(gedi_masked)

In [8]:
Map.addLayer(image.select('agbd'),{},"agbd")

Map

Map(bottom=141908.0, center=[-14.317614840171844, 27.661514282226566], controls=(WidgetControl(options=['posit…

In [9]:
import time

sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
        scale=30,
        region=roi,
        geometries=True
    )

export_task = ee.batch.Export.table.toDrive(
        collection=sample,
        description='ExportSampleToCSV',
        folder='Zambia_draw_3',
        fileNamePrefix=f'Datos_RF_2020',
        fileFormat='CSV'
    )

# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1756319447129, 'update_timestamp_ms': 1756319477244, 'start_timestamp_ms': 1756319456987, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 0.15884836, 'id': 'O7TX4Q2ZDINFAU3ESFZBBBT4', 'name': 'projects/ee-facuboladerasgee/operations/O7TX4Q2ZDINFAU3ESFZBBBT4'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1756319447129, 'update_timestamp_ms': 1756319507233, 'start_timestamp_ms': 1756319456987, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 1722.832519531, 'id': 'O7TX4Q2ZDINFAU3ESFZBBBT4', 'name': 'projects/ee-facuboladerasgee/operations/O7TX4Q2ZDINFAU3ESFZBBBT4'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 1

# Predicts

In [23]:
feature_names = ['A39','A61','A01','A48','A36','A51','A30','A47','A50','A21']

image = embedding_image1.select(feature_names)
image2 = embedding_image2.select(feature_names)

In [24]:
feature_names = ['A39','A61','A01','A48','A36','A51','A30','A47','A50','A21']

label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/zambia_balanced"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)
classified2 = image2.select(feature_names).classify(another_classifier)

In [28]:
DW = 'GOOGLE/DYNAMICWORLD/V1'

# DW para 2024
dw2024 = (ee.ImageCollection(DW)
          .filterDate('2022-01-01', '2024-12-31')
          .filterBounds(roi)
          .select('label')
          .mode()
          .clip(roi))


mask2024 = dw2024.neq(0)

# ---------------------------------------------------------------------
# 3) Aplicar máscara a tus clasificaciones
# ---------------------------------------------------------------------
classified_masked  = classified.updateMask(mask2024)
classified2_masked = classified2.updateMask(mask2024)

In [29]:
visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}


Map.centerObject(roi)



Map.addLayer(
    classified_masked,
    visPredictedBiomass,
    "im1",)

Map.addLayer(
    classified2_masked,
    visPredictedBiomass,
    "im2",)


Map

Map(bottom=4589.0, center=[-8.450638800331001, 33.64013671875001], controls=(WidgetControl(options=['position'…

In [30]:
# Export 2017
task1 = ee.batch.Export.image.toDrive(
    image=classified_masked,
    description='Prediccion_2019',
    folder='Zambia_random',       # Carpeta en tu Google Drive
    fileNamePrefix='Prediccion_2017',
    region=roi,
    scale=100,                    # Resolución de 100 metros
    crs='EPSG:4326',              # Puedes ajustar el CRS según tus datos
    maxPixels=1e13
)
task1.start()

# Export 2024
task2 = ee.batch.Export.image.toDrive(
    image=classified2_masked,
    description='Prediccion_2024',
    folder='Zambia_random',
    fileNamePrefix='Prediccion_2024',
    region=roi,
    scale=100,
    crs='EPSG:4326',
    maxPixels=1e13
)
task2.start()

print("🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE")

🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE
